In [18]:
#Import Librairies

import pandas as pd
import numpy as np
import os
import glob
import csv
#import Selenium for webscraping
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
%matplotlib inline
sns.set(style= 'darkgrid')
sns.set_palette('deep')


#transformations
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Perceptron
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn import metrics

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import precision_score, recall_score

In [19]:
#read data books.csv
df0 = pd.read_csv('books.csv', sep=';')

In [20]:
#show data unique values 
df0.nunique()




bookID                11127
title                 10350
authors                6625
average_rating          212
isbn                  11075
isbn13                  242
language_code            30
  num_pages             997
ratings_count          5264
text_reviews_count     1807
publication_date       3675
publisher              2285
Unnamed: 12               4
dtype: int64

In [21]:
#Num_pages and isbn13 are object while they should be int
#the easiest column to check the issue is language_code as got less unique values than the others

#count values of language_code
df0['language_code'].value_counts()

eng            8859
en-US          1407
spa             218
en-GB           214
fre             144
ger              99
jpn              46
mul              19
zho              14
grc              11
por              10
en-CA             7
ita               5
lat               3
enm               3
rus               2
swe               2
ara               1
wel               1
gla               1
tur               1
9,78085E+12       1
nor               1
9,78156E+12       1
glg               1
msa               1
nl                1
9,78159E+12       1
9,78067E+12       1
ale               1
Name: language_code, dtype: int64

In [22]:
#Now we clearly visualize that the issue is due to some values being corresponding to isbn13 instead of language_code. 4 lines are impacted.
#locate the rows for 9,78085E+12,9,78156E+12,9,78159E+12 and 9,78067E+12
df0.loc[df0['language_code'].isin (['9,78085E+12', '9,78156E+12', '9,78159E+12', '9,78067E+12'])]


,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,Unnamed: 12
3348,12224,Streetcar Suburbs: The Process of Growth in Bo...,Sam Bass Warner,Jr./Sam B. Warner,3.58,674842111,"9,78067E+12",en-US,236.0,61.0,6,4/20/2004,Harvard University Press
4702,16914,The Tolkien Fan's Medieval Reader,David E. Smith (Turgon of TheOneRing.net,one of the founding members of this Tolkien w...,3.58,1593600119,"9,78159E+12",eng,400.0,26.0,4,04/06/2004,Cold Spring Press
5877,22128,Patriots (The Coming Collapse),James Wesley,Rawles,3.63,156384155X,"9,78156E+12",eng,342.0,38.0,4,1/15/1999,Huntington House Publishers
8979,34889,Brown's Star Atlas: Showing All The Bright Sta...,Brown,Son & Ferguson,0.00,851742718,"9,78085E+12",eng,49.0,0.0,0,05/01/1977,Brown Son & Ferguson Ltd.


In [23]:
#Clearly now we can see that the column authors was splitted in 2 columns and its values were assigned to average_rating

#rows 3348,4702,5877 and 8979 need to be moved by 1 column to the left
df0.iloc[[3348, 4702, 5877, 8979], 3:13] = df0.iloc[[3348, 4702, 5877, 8979], 3:13].shift(periods=-1, axis="columns")


In [24]:
#now let's visualize if the issue is solved
df0.iloc[[3348, 4702, 5877, 8979]]

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,Unnamed: 12
3348,12224,Streetcar Suburbs: The Process of Growth in Bo...,Sam Bass Warner,3.58,674842111,"9,78067E+12",en-US,236.0,61.0,6,4/20/2004,Harvard University Press,NaN
4702,16914,The Tolkien Fan's Medieval Reader,David E. Smith (Turgon of TheOneRing.net,3.58,1593600119,"9,78159E+12",eng,400.0,26.0,4,04/06/2004,Cold Spring Press,NaN
5877,22128,Patriots (The Coming Collapse),James Wesley,3.63,156384155X,"9,78156E+12",eng,342.0,38.0,4,1/15/1999,Huntington House Publishers,NaN
8979,34889,Brown's Star Atlas: Showing All The Bright Sta...,Brown,0.00,851742718,"9,78085E+12",eng,49.0,0.0,0,05/01/1977,Brown Son & Ferguson Ltd.,NaN


In [25]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bookID              11127 non-null  int64  
 1   title               11127 non-null  object 
 2   authors             11076 non-null  object 
 3   average_rating      11076 non-null  object 
 4   isbn                11076 non-null  object 
 5   isbn13              11076 non-null  object 
 6   language_code       11076 non-null  object 
 7     num_pages         11076 non-null  object 
 8   ratings_count       11076 non-null  float64
 9   text_reviews_count  11076 non-null  object 
 10  publication_date    11076 non-null  object 
 11  publisher           11076 non-null  object 
 12  Unnamed: 12         0 non-null      object 
dtypes: float64(1), int64(1), object(11)
memory usage: 1.1+ MB


In [26]:
#remove empty space in column num_pages
df0 = df0.rename(columns={"  num_pages": "num_pages"})

In [27]:
#visualize the NaN values in the dataset
df0.isnull().sum()

bookID                    0
title                     0
authors                  51
average_rating           51
isbn                     51
isbn13                   51
language_code            51
num_pages                51
ratings_count            51
text_reviews_count       51
publication_date         51
publisher                51
Unnamed: 12           11127
dtype: int64

In [28]:
#drop the column unnamed: 12
df0.drop('Unnamed: 12', axis=1, inplace=True)

#display the rows with NaN values limit to 10 rows
df0[df0.isnull().any(axis=1)].head(10)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
483,1537,The Oedipus Plays of Sophocles: Oedipus the King,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
490,1549,Antigone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
751,2442,Witches Abroad (Discworld #12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1028,3479,Sugarplums and Scandal (Love at Stake #2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1155,3852,Primal Heat (Includes: Breeds #8.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1643,5720,Rainbow Six (John Clark #2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1705,5934,Veinte poemas de amor y una canción desesperada,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1866,6625,Playing Easy to Get (B.A.D. Agency #1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2192,7848,Moby-Dick,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2758,10200,Venac sonetnih venaca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
#drop the rows with NaN values
df0.dropna(inplace=True)

In [30]:
#convert isbn13 to int, num_pages and text_reviews_count to int and average_rating to float
df0 = df0.astype({"average_rating": float, "num_pages": int, "text_reviews_count": int,"ratings_count": int})

In [31]:
#convert isbn13 to int
df0['isbn13'] = df0['isbn13'].astype(int)

ValueError: invalid literal for int() with base 10: '9,78044E+12'

In [32]:
df0.head(4)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,439785960,"9,78044E+12",eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,439358078,"9,78044E+12",eng,870,2153167,29221,09/01/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,439554896,"9,78044E+12",eng,352,6333,244,11/01/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,"9,78044E+12",eng,435,2339585,36325,05/01/2004,Scholastic Inc.


In [33]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11076 entries, 0 to 11126
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   bookID              11076 non-null  int64  
 1   title               11076 non-null  object 
 2   authors             11076 non-null  object 
 3   average_rating      11076 non-null  float64
 4   isbn                11076 non-null  object 
 5   isbn13              11076 non-null  object 
 6   language_code       11076 non-null  object 
 7   num_pages           11076 non-null  int32  
 8   ratings_count       11076 non-null  int32  
 9   text_reviews_count  11076 non-null  int32  
 10  publication_date    11076 non-null  object 
 11  publisher           11076 non-null  object 
dtypes: float64(1), int32(3), int64(1), object(7)
memory usage: 995.1+ KB


In [ ]:
#as isbn13 cannot get converted to int, I need to work around it and do it manually as I will need this data to search for the books categories (genres) on goodreads.com

#save file to csv
df0.to_csv('books_original_file_csv.csv', sep=';')

In [ ]:
#Load the books original dataset 
df0 = pd.read_csv("books_original_file.csv",sep=";")

In [ ]:
#I decided to add the categories of the books to get a richer approach through web scraping the categories (genres) on goodreads.
# Webscraping with Selenium to get the categories of the books from the isbn13 on the website goodreads.com

options = Options()
#options.add_argument('headless')
website='https://www.goodreads.com'
path=r"C:\Users\sophi\OneDrive\Bureau\python_project\chromedriver.exe"
driver=webdriver.Chrome(path,chrome_options=options)
#acceptedcookies=False

part="_part9.csv"
def autosearch(isbn13):
   
    #global acceptedcookies
    
   
    driver.get(website)
    driver.implicitly_wait(5)
   
    
    searchbar=driver.find_elements(By.ID,'sitesearch_field')[0]
    searchbuttonposition=driver.find_elements(By.CLASS_NAME,'submitLink')[0]

    searchbutton=searchbuttonposition.find_elements(By.TAG_NAME,'img')[0]
    searchbar.send_keys(isbn13[:-1])
    searchbutton.click()
    driver.implicitly_wait(8)
    try:
        genrelist=driver.find_elements(By.CLASS_NAME, 'elementList')
        li_list=genrelist[0].find_elements(By.CLASS_NAME, 'left')[0]
        categories=li_list.find_elements(By.TAG_NAME, 'a')[0].text
    #deduplicated=max(set(categories), key = categories.count)
        print(categories)
        
        
        return(categories)
    except:
        print('NA')
        return('NA')
    
bookDF=pd.read_csv(r"C:\Users\sophi\DSTI\Project Python Lab\Project DA\Books_for_analysis.csv +part",sep=";")
print(bookDF.columns)

#create a new column + iterate for each line with the above function
df0['categories']=df0['isbn13'].apply(lambda x:autosearch(str(x)))

In [ ]:
#Quite the driver once the scraping is done
driver.quit()

In [ ]:
#save the dataset with the new column categories
df0.to_csv('file_4_prediction_csv+part', sep=';')

In [ ]:
#11K calls on a website is a lot and can be interrupted at any moment, so I split the dataset in 12 parts then I concatenated them into 1 single dataset

os.chdir(r"C:\Users\sophi\DSTI\Project Python Lab\Project DA\books_for_prediction_ALL")
extension = 'csv'
all_filenames = glob.glob('*.{}'.format(extension))
#combine all files in the list
file_4_prediction_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
file_4_prediction_csv.to_csv( "file_4_prediction_csv.csv", index=False,encoding='UTF-8')
print(all_filenames)